# Generate Mock X.509 Certificates

In [1]:
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.serialization import Encoding, PrivateFormat, NoEncryption
from cryptography.hazmat.backends import default_backend
from datetime import datetime, timedelta
import os

In [2]:
def save_pem_file(filename, pem_data):
    with open(filename, 'wb') as f:
        f.write(pem_data)

def generate_private_key(key_size=2048):
    return rsa.generate_private_key(public_exponent=65537, key_size=key_size)

# Generate self-signed CA certificate
def generate_ca_certificate(private_key):
    subject = issuer = x509.Name([
        x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
        x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"NY"),
        x509.NameAttribute(NameOID.LOCALITY_NAME, u"New York"),
        x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"Mock CA Organization"),
        x509.NameAttribute(NameOID.COMMON_NAME, u"Mock CA"),
    ])

    ca_certificate = x509.CertificateBuilder(
    ).subject_name(
        subject
    ).issuer_name(
        issuer
    ).public_key(
        private_key.public_key()
    ).serial_number(
        x509.random_serial_number()
    ).not_valid_before(
        datetime.utcnow()
        # datetime.utcnow() - timedelta(days=3650) # Expired CA
    ).not_valid_after(
        datetime.utcnow() + timedelta(days=3650) # CA is valid for 10 years
        # datetime.utcnow() - timedelta(days=365) # Expired CA
    ).add_extension(
        x509.BasicConstraints(ca=True, path_length=0), critical=True, # CA: TRUE
        # x509.BasicConstraints(ca=False, path_length=None), critical=True, # CA: FALSE
    ).sign(private_key, hashes.SHA256(), default_backend())

    return ca_certificate

# Generate an end-entity certificate signed by the CA
def generate_end_entity_certificate(private_key, ca_certificate, ca_private_key):
    subject = x509.Name([
        x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
        x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"NY"),
        x509.NameAttribute(NameOID.LOCALITY_NAME, u"New York"),
        x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"Mock Organization"),
        x509.NameAttribute(NameOID.COMMON_NAME, u"www.mock.com"),
    ])

    end_entity_certificate = x509.CertificateBuilder(
    ).subject_name(
        subject
    ).issuer_name(
        # ca_certificate.subject # Correct issuer
        subject # Incorrect issuer
    ).public_key(
        private_key.public_key()
    ).serial_number(
        x509.random_serial_number()
    ).not_valid_before(
        datetime.utcnow()
        # datetime.strptime("2024-02-19 14:24:14 +0000 UTC", "%Y-%m-%d %H:%M:%S %z %Z")  # valid from
    ).not_valid_after(
        # datetime.utcnow() + timedelta(days=365) # valid for 1 year
        # datetime.strptime("2024-03-19 14:24:14 +0000 UTC", "%Y-%m-%d %H:%M:%S %z %Z") # expires on
        datetime.strptime("5024-03-19 14:24:14 +0000 UTC", "%Y-%m-%d %H:%M:%S %z %Z") # expires in eons
    ).add_extension(
        x509.BasicConstraints(ca=False, path_length=None), critical=True,
    ).add_extension(
        x509.KeyUsage(digital_signature=True, key_encipherment=True, content_commitment=False, data_encipherment=False,
                      key_agreement=False, key_cert_sign=False, crl_sign=False, encipher_only=False, decipher_only=False), critical=True,
    ).add_extension(
        x509.ExtendedKeyUsage([x509.ExtendedKeyUsageOID.SERVER_AUTH]), critical=True,
    ).sign(ca_private_key, hashes.SHA256(), default_backend())

    return end_entity_certificate

# Save private key and certificate to PEM files
def save_private_key_to_pem(private_key, filename):
    pem_data = private_key.private_bytes(
        encoding=Encoding.PEM,
        format=PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=NoEncryption()
    )
    save_pem_file(filename, pem_data)


def save_certificate_to_pem(certificate, filename):
    pem_data = certificate.public_bytes(Encoding.PEM)
    save_pem_file(filename, pem_data)

In [3]:
KEY_SIZE = 4096

# Generate CA private key
ca_private_key = generate_private_key(KEY_SIZE)

# Generate CA certificate
ca_certificate = generate_ca_certificate(ca_private_key)

# Save CA private key and certificate
save_private_key_to_pem(ca_private_key, './out/ca_key.pem')
save_certificate_to_pem(ca_certificate, './out/ca_cert.pem')

# Generate end-entity private key
end_entity_private_key = generate_private_key(KEY_SIZE)

# Generate end-entity certificate signed by the CA
end_entity_certificate = generate_end_entity_certificate(
    end_entity_private_key, ca_certificate, ca_private_key)

# Save end-entity private key and certificate
save_private_key_to_pem(end_entity_private_key, './out/cert_key.pem')
save_certificate_to_pem(end_entity_certificate, './out/cert.pem')

print("CA and end-entity certificates generated successfully")

CA and end-entity certificates generated successfully
